# 作业2 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [2]:
import pandas as pd

# 1. 从fundemantals.csv开始！

fundemantals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [3]:
fund = pd.read_csv(r'F:\kaggle数据集\fundamentals.csv')
df_fund = pd.DataFrame(fund)

#第一问
net_income = df_fund['Net Income'].groupby([df_fund['For Year']])
mean = net_income.mean()    #得到2015年net income的平均值
print('2015年net income的平均值：',mean[2015.0])
dvalue = net_income.max()-net_income.min()  #求最大值与最小值之差
print('最大值与最小值的差值：',dvalue[2015.0])

#第二问
df_fund['Fixed/Total'] = df_fund['Fixed Assets']/df_fund['Total Assets']  #添加一列用于记录固定资产与总资产的比
f_t = df_fund.groupby('For Year')['Fixed/Total']
f_t_mean = f_t.mean()  #得到固定资产与总资产比值的平均值
print('2016年固定资产占总资产比例的均值是{:.5%}'.format(f_t_mean[2016.0]))

fixed_ticker = df_fund.groupby('Ticker Symbol')['Fixed Assets']
total_ticker = df_fund.groupby('Ticker Symbol')['Total Assets']
f_t_proportion = fixed_ticker.sum()/total_ticker.sum()  #求得固各股票代码的总固定与总资产的比值
get_index = pd.DataFrame(f_t_proportion.sort_values(),columns = ['proportion'])  #创建比值的DataFrame
min_index = get_index[get_index.proportion == get_index.proportion.min()].index  #得到比值最小的股票代码
print('固定资产占总资产比例最小的股票代码是{}'.format(list(min_index)))

2015年net income的平均值： 1575185571.764706
最大值与最小值的差值： 76922000000.0
2016年固定资产占总资产比例的均值是18.55779%
固定资产占总资产比例最小的股票代码是['PRU', 'MET', 'TMK', 'TRV', 'XL', 'C', 'CB', 'AMP', 'PNC', 'AIG', 'SYF', 'LEN', 'NAVI']


# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [6]:
#导入csv
sec = pd.read_csv(r'F:\kaggle数据集\securities.csv')
df_sec = pd.DataFrame(sec)

sortedsec = sec.sort_values(by = ['GICS Sector','Date first added'])  #将文件按照GICS Sector和Date first added排序
sectors = list(sortedsec['GICS Sector'].unique())  #得到所有的sector，生成的是ndarray，转成list
ticker = []  #存放每个sector中加入时间最早的ticker

#每个sector中加入时间最早的ticker可能不止一个
for sector in sectors:
    each_first = [sortedsec[sortedsec['GICS Sector']==sector]['Ticker symbol'].iloc[0]]  #存放每个sector中加入时间最早的ticker
    isfirst = True  
    index = 1
    while isfirst:
        #判断第index个时间是否是最早时间
        if sortedsec[sortedsec['GICS Sector']==sector]['Date first added'].iloc[0]==sortedsec[sortedsec['GICS Sector']==sector]['Date first added'].iloc[index]:
            each_first.append(sortedsec[sortedsec['GICS Sector']==sector]['Ticker symbol'].iloc[index])
            index += 1
        else:
            isfirst = False
    ticker.append(each_first)
first_ticker = dict(zip(sectors,ticker))
[print('{}中加入时间最早的ticker是{}'.format(key,value)) for key,value in first_ticker.items()]

Consumer Discretionary中加入时间最早的ticker是['MCD']
Consumer Staples中加入时间最早的ticker是['GIS', 'CLX']
Energy中加入时间最早的ticker是['SLB']
Financials中加入时间最早的ticker是['JPM']
Health Care中加入时间最早的ticker是['ABT']
Industrials中加入时间最早的ticker是['ARNC', 'HON']
Information Technology中加入时间最早的ticker是['HPQ']
Materials中加入时间最早的ticker是['SHW']
Real Estate中加入时间最早的ticker是['AVB']
Telecommunications Services中加入时间最早的ticker是['T', 'VZ']
Utilities中加入时间最早的ticker是['DUK', 'NEE']


[None, None, None, None, None, None, None, None, None, None, None]

In [8]:
#就是用同上的方法修改一下参数
sortedsec = sec.sort_values(by = ['Address of Headquarters','Date first added'],ascending = [True,False])
sectors_address = list(sortedsec['Address of Headquarters'].unique())
ticker_address = []
for sector in sectors_address:
    each_last = [sortedsec[sortedsec['Address of Headquarters']==sector]['Ticker symbol'].iloc[0]]
    islast = True
    index = 1
    while isfirst:
        if sortedsec[sortedsec['Address of Headquarters']==sector]['Date first added'].iloc[0]==sortedsec[sortedsec['Address of Headquarters']==sector]['Date first added'].iloc[index]:
            each_last.append(sortedsec[sortedsec['Address of Headquarters']==sector]['Ticker symbol'].iloc[index])
            index += 1
        else:
            islast = False
    ticker_address.append(each_last)
last_ticker = dict(zip(sectors_address,ticker_address))
for key,value in last_ticker.items():
    print('{}中加入时间最晚的ticker是{}'.format(key,value))

Akron, Ohio中加入时间最晚的ticker是['FE']
Allentown, Pennsylvania中加入时间最晚的ticker是['APD']
Amsterdam, Netherlands中加入时间最晚的ticker是['MYL']
Amsterdam, New York中加入时间最晚的ticker是['MHK']
Arlington, Virginia中加入时间最晚的ticker是['AES']
Arlington, Virginia[3]中加入时间最晚的ticker是['AVB']
Armonk, New York中加入时间最晚的ticker是['IBM']
Atlanta, Georgia中加入时间最晚的ticker是['AYI']
Auburn Hills, Michigan中加入时间最晚的ticker是['BWA']
Austin, Minnesota中加入时间最晚的ticker是['HRL']
Austin, Texas中加入时间最晚的ticker是['WFM']
Baltimore, Maryland中加入时间最晚的ticker是['UAA']
Baton Rouge, Louisiana中加入时间最晚的ticker是['ALB']
Battle Creek, Michigan中加入时间最晚的ticker是['K']
Bellevue, Washington中加入时间最晚的ticker是['EXPE']
Benton Harbor, Michigan中加入时间最晚的ticker是['WHR']
Bentonville, Arkansas中加入时间最晚的ticker是['WMT']
Berwyn, Pennsylvania中加入时间最晚的ticker是['AME']
Bethesda, Maryland中加入时间最晚的ticker是['HST']
Beverly, Massachusetts中加入时间最晚的ticker是['AMG']
Birmingham, Alabama中加入时间最晚的ticker是['RF']
Boise, Idaho中加入时间最晚的ticker是['MU']
Bolingbrook, Illinois中加入时间最晚的ticker是['ULTA']
Boston, Massachusetts中加入时间最晚的ticker

# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

In [14]:
df_sec = df_sec.rename(columns={'Ticker symbol':'Ticker Symbol'})  #两个csv的名字不对，修改一下名字
df_merge = pd.merge(df_fund,df_sec,how = 'inner',on = 'Ticker Symbol')  #按照ticker symbol为key以inner方式合并
merge_2013 = df_merge[df_merge['For Year']>=2013]  #先筛选出>=2013年的所有数据
merge_2013_2016 = merge_2013[merge_2013['For Year']<=2016]  #在>=2013年的数据中选出<=2016年的数据
total = dict(merge_2013_2016['Research and Development'].groupby(merge_2013_2016['GICS Sector']).sum())
[print('{}在2013-2016年Research and Development的总投入为{}'.format(key,value)) for key,value in total.items()]

Consumer Discretionary在2013-2016年Research and Development的总投入为1860178000.0
Consumer Staples在2013-2016年Research and Development的总投入为1346700000.0
Energy在2013-2016年Research and Development的总投入为1692513000.0
Financials在2013-2016年Research and Development的总投入为0.0
Health Care在2013-2016年Research and Development的总投入为152722357000.0
Industrials在2013-2016年Research and Development的总投入为42622899000.0
Information Technology在2013-2016年Research and Development的总投入为281823342000.0
Materials在2013-2016年Research and Development的总投入为18509206000.0
Real Estate在2013-2016年Research and Development的总投入为116210000.0
Telecommunications Services在2013-2016年Research and Development的总投入为0.0
Utilities在2013-2016年Research and Development的总投入为0.0


[None, None, None, None, None, None, None, None, None, None, None]

In [15]:
#降序排列分组后生成的按照ticker symbol分组的求和数据
invest_ticker = merge_2013_2016['Research and Development'].groupby(merge_2013_2016['Ticker Symbol']).sum().sort_values(ascending = False)
max_3 = list(invest_ticker.index[:3])
print('在2013年和2016年对于Research and Development投入最大的三家公司是{}'.format(max_3))

在2013年和2016年对于Research and Development投入最大的三家公司是['MSFT', 'INTC', 'AAPL']


# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？